In [8]:
# Gerekli Kütüphanelerin Yüklenmesi
import pandas as pd
import numpy as np
import random
import json
import io
import os
from PIL import Image
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import NearestNeighbors

# GENAI IMPORT
try:
    import google.generativeai as genai
    HAS_GENAI = True
except ImportError:
    HAS_GENAI = False
    print("⚠️ Uyarı: 'google-generativeai' kütüphanesi bulunamadı.")

# API KEY YÖNETİMİ (CONFIG DESTEKLİ)
API_KEY = os.getenv("GEMINI_API_KEY")

if not API_KEY:
    try:
        import config
        API_KEY = config.MY_KEY
        print("🔑 Başarılı: API Key 'config.py' dosyasından yüklendi.")
    except ImportError:
        pass

# Bağlantı Kontrolü
if API_KEY and HAS_GENAI:
    try:
        genai.configure(api_key=API_KEY)
        print("✅ Bağlantı: Canlı Gemini Modu Aktif.")
        
                
    except Exception as e:
        print(f"⚠️ Hata: API Key geçersiz veya bağlantı sorunu var. ({e})")
        API_KEY = None 
else:
    print("\nℹ️ Bilgi: API Key bulunamadı. Sistem 'Offline Mock Modunda' çalışacak.")


# 3. MOCK DATA (ÇEVRİMDIŞI SENARYO İÇİN)
MOCK_RESULTS = {
    "test_image.jpg": {
        "Product_Type": "Trousers",
        "Color": "Black",
        "Style": "Casual",
        "Gender": "Men",
        "Season": "Autumn",
        "Pattern": "Solid",
        "Price": 0,
        "Review_Rating": 0
    },
    "default": { 
        "Product_Type": "T-Shirt",
        "Color": "White",
        "Style": "Casual",
        "Gender": "Unisex"
    }
}

🔑 Başarılı: API Key 'config.py' dosyasından yüklendi.
✅ Bağlantı: Canlı Gemini Modu Aktif.


In [9]:
# VERİ SETİ
DATA_FILE = 'fashion_products_mock_data.csv'

# 1. KONTROL: Dosya zaten var mı?
if os.path.exists(DATA_FILE):
    print(f"✅ '{DATA_FILE}' bulundu. Veriler diskten yükleniyor... (Zaman kazandık!)")
    df = pd.read_csv(DATA_FILE)
    print(f"Veri Seti Boyutu: {df.shape}")
    
else:
    # 2. ÜRETİM: Dosya yoksa sıfırdan üret
    print("⚠️ Dosya bulunamadı. Mock Data sıfırdan üretiliyor... (Bu işlem biraz sürebilir)")
    
    NUM_ROWS = 20000
    PRODUCT_TYPES = [
        'T-Shirt', 'Blouse', 'Shirt', 'Tank Top', 'Hoodie', 'Sweater', 'Cardigan',
        'Jacket', 'Coat', 'Blazer', 'Vest', 'Jeans', 'Trousers', 'Shorts', 'Skirt',
        'Dress', 'Jumpsuit', 'Swimwear', 'Bikini', 'Underwear', 'Socks', 'Sneakers',
        'Boots', 'Loafers', 'Heels', 'Sandals', 'Slippers', 'Cap', 'Hat', 'Scarf'
    ]

    # Features
    COLORS = ['Black', 'White', 'Red', 'Blue', 'Green', 'Yellow', 'Purple', 'Pink', 'Grey', 'Beige', 'Navy', 'Orange', 'Brown', 'Multi', 'Silver', 'Gold']
    MATERIALS = ['Cotton', 'Polyester', 'Wool', 'Silk', 'Linen', 'Leather', 'Denim', 'Velvet', 'Nylon', 'Rayon', 'Spandex', 'Cashmere', 'Suede', 'Viscose']
    PATTERNS = ['Solid', 'Striped', 'Polka Dot', 'Floral', 'Plaid', 'Geometric', 'Animal Print', 'Camouflage', 'Tie Dye', 'Graphic', 'Checkered']
    STYLES = ['Casual', 'Formal', 'Sporty', 'Bohemian', 'Vintage', 'Chic', 'Streetwear', 'Business', 'Minimalist', 'Gothic', 'Preppy']
    SEASONS = ['Spring', 'Summer', 'Autumn', 'Winter', 'All Season']
    GENDERS = ['Men', 'Women', 'Unisex', 'Kids']
    BRANDS = [f'Brand_{i}' for i in range(1, 51)] 
    SIZES = ['XXS', 'XS', 'S', 'M', 'L', 'XL', 'XXL', '3XL', 'One Size']
    FITS = ['Slim', 'Regular', 'Loose', 'Oversized', 'Skinny', 'Tailored']
    NECKLINES = ['Round', 'V-Neck', 'Collared', 'Turtle Neck', 'Off Shoulder', 'Scoop', 'Square', 'Halter', 'Hooded', 'None']
    SLEEVES = ['Short', 'Long', 'Sleeveless', '3/4 Sleeve', 'Cap Sleeve', 'None']

    def generate_mock_data(n_rows):
        data = []
        
        for i in range(n_rows):
            prod_type = random.choice(PRODUCT_TYPES)
            
            is_footwear = prod_type in ['Sneakers', 'Boots', 'Loafers', 'Heels', 'Sandals', 'Slippers']
            is_accessory = prod_type in ['Cap', 'Hat', 'Scarf', 'Socks']
            
            row = {
                'Product_ID': f'PROD_{10000+i}',
                'Product_Type': prod_type,
                'Brand': random.choice(BRANDS),
                'Price': round(random.uniform(10.0, 500.0), 2),
                'Currency': 'USD',
                'Color': random.choice(COLORS),
                'Material': random.choice(MATERIALS),
                'Pattern': random.choice(PATTERNS),
                'Style': random.choice(STYLES),
                'Season': random.choice(SEASONS),
                'Gender': random.choice(GENDERS),
                'Size': random.choice(SIZES),
                'Fit': 'None' if (is_footwear or is_accessory) else random.choice(FITS),
                'Neckline': 'None' if (is_footwear or is_accessory) else random.choice(NECKLINES),
                'Sleeve_Length': 'None' if (is_footwear or is_accessory) else random.choice(SLEEVES),
                'Occasion': random.choice(['Daily', 'Party', 'Gym', 'Work', 'Beach', 'Wedding']),
                'Review_Rating': round(random.uniform(1.0, 5.0), 1),
                'Review_Count': random.randint(0, 5000),
                'Is_Sustainable': random.choice([True, False]),
                'Has_Pockets': random.choice([True, False]),
                'Is_Waterproof': random.choice([True, False]),
                'Is_Machine_Washable': random.choice([True, False]),
                'Has_Zipper': random.choice([True, False]),
                'Has_Buttons': random.choice([True, False]),
                'Is_Iron_Free': random.choice([True, False]),
                'Is_Limited_Edition': random.choice([True, False]),
                'Is_On_Sale': random.choice([True, False]),
                'Has_Hood': True if prod_type in ['Hoodie', 'Jacket', 'Coat'] and random.random() > 0.5 else False,
                'Is_Handmade': random.choice([True, False]),
                'Is_Organic': random.choice([True, False]),
                'Has_Lining': random.choice([True, False]),
                'Is_Stretch': random.choice([True, False]),
                'Has_Embroidery': random.choice([True, False]),
                'Is_Quick_Dry': random.choice([True, False]),
                'Has_Belt': random.choice([True, False]),
                'Is_Transparent': random.choice([True, False]),
                'Has_Reflective_Details': random.choice([True, False]),
                'Is_Vegan': random.choice([True, False]),
                'Weight_Grams': random.randint(100, 2000),
                'Origin_Country': random.choice(['Turkey', 'China', 'Italy', 'Vietnam', 'India', 'USA']),
                'Stock_Level': random.randint(0, 1000),
                'Days_Since_Release': random.randint(1, 730),
                'Warranty_Months': random.choice([0, 6, 12, 24]),
                'Returnable': random.choice([True, False]),
                'Shipping_Weight': round(random.uniform(0.1, 3.0), 2),
                'Package_Width_cm': random.randint(10, 50),
                'Package_Height_cm': random.randint(2, 20),
                'Package_Length_cm': random.randint(15, 60),
                'SKU_Code': f'SKU_{random.randint(100000, 999999)}',
                'Care_Instructions': random.choice(['Machine Wash', 'Dry Clean Only', 'Hand Wash']),
                'Target_Audience': random.choice(['Teens', 'Adults', 'Seniors']),
                'Collection_Year': random.choice([2023, 2024, 2025, 2026])
            }
            data.append(row)
            
        return pd.DataFrame(data)

    # Veriyi Üret
    print("Veri üretiliyor...")
    df = generate_mock_data(NUM_ROWS)
    
    # CSV olarak kaydet
    df.to_csv(DATA_FILE, index=False)
    print(f"✅ Veri üretildi ve '{DATA_FILE}' dosyasına kaydedildi.")
    print(f"Veri Seti Boyutu: {df.shape}")

# Son Kontrol
print("\nİlk 5 Satır:")
print(df.head())

✅ 'fashion_products_mock_data.csv' bulundu. Veriler diskten yükleniyor... (Zaman kazandık!)
Veri Seti Boyutu: (20000, 52)

İlk 5 Satır:
   Product_ID Product_Type     Brand   Price Currency  Color   Material  \
0  PROD_10000       Bikini   Brand_1  211.58      USD  White  Polyester   
1  PROD_10001       Jacket  Brand_38  214.04      USD   Grey      Denim   
2  PROD_10002        Shirt  Brand_10  336.88      USD  Beige      Linen   
3  PROD_10003       Blazer  Brand_22  211.39      USD  Black      Nylon   
4  PROD_10004     Jumpsuit   Brand_8  251.35      USD   Navy      Nylon   

      Pattern     Style      Season  ... Warranty_Months Returnable  \
0   Geometric    Formal  All Season  ...               0      False   
1  Camouflage  Bohemian      Spring  ...               0      False   
2  Camouflage    Gothic      Summer  ...               0      False   
3     Striped  Bohemian      Summer  ...               6      False   
4     Tie Dye   Vintage      Spring  ...               0  

In [10]:
# MODEL SINIFI

class FashionRecommender:
    def __init__(self, data, weights=None):
        self.df = data.copy()
        self.model = None
        self.preprocessor = None
        self.features_matrix = None
        
        # Ağırlıklar
        self.weights = weights if weights else {
            'Product_Type': 5.0,
            'Color': 2.0,
            'Style': 1.0,
            'Price': 1.0,
            'Brand': 1.0
        }
        
        # Tamamlayıcı Ürün Haritası
        self.complementary_map = {
            'T-Shirt': ['Jeans', 'Shorts', 'Skirt', 'Sweatpants'],
            'Blouse': ['Skirt', 'Trousers', 'Shorts'],
            'Shirt': ['Trousers', 'Jeans', 'Blazer'],
            'Hoodie': ['Sweatpants', 'Jeans', 'Sneakers'],
            'Jeans': ['T-Shirt', 'Shirt', 'Sweater', 'Sneakers', 'Belt'],
            'Dress': ['Heels', 'Sandals', 'Jewelry', 'Bag'],
            'Sneakers': ['Socks', 'Cap', 'Backpack', 'T-Shirt'],
            'Trousers': ['Shirt', 'T-Shirt', 'Sweater', 'Belt', 'Sneakers'],
        }
        
        self._build_model()

    def _build_model(self):
        """Veriyi işler, ağırlıklandırır ve KNN modelini eğitir."""
        
        categorical_features = ['Product_Type', 'Color', 'Material', 'Pattern', 
                                'Style', 'Season', 'Gender', 'Occasion', 'Fit']
        numerical_features = ['Price', 'Review_Rating'] 

        # Pipeline
        self.preprocessor = ColumnTransformer(
            transformers=[
                ('num', MinMaxScaler(), numerical_features),
                ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
            ],
            verbose_feature_names_out=True
        )

        print("Veriler vektörleştiriliyor...")
        transformed_data = self.preprocessor.fit_transform(self.df)
        feature_names = self.preprocessor.get_feature_names_out()
        
        # Ağırlıklandırma
        print("Özellik ağırlıkları uygulanıyor...")
        weighted_data = transformed_data.copy()
        
        for feature_name, weight in self.weights.items():
            indices = [i for i, name in enumerate(feature_names) if feature_name in name]
            if indices:
                weighted_data[:, indices] = weighted_data[:, indices] * weight
        
        self.features_matrix = weighted_data
        
        # Model Eğitimi
        self.model = NearestNeighbors(n_neighbors=2000, metric='cosine', algorithm='brute')
        self.model.fit(self.features_matrix)
        print("Model eğitildi ve hazır!")

    def get_similar_products(self, product_id, n_recommendations=15):
        """Seçilen ürüne en benzer N ürünü getirir."""
        try:
            idx = self.df.index[self.df['Product_ID'] == product_id].tolist()[0]
        except IndexError:
            return pd.DataFrame()

        # DÜZELTME 1: .reshape(1, -1) eklendi
        query_vector = self.features_matrix[idx].reshape(1, -1)
        distances, indices = self.model.kneighbors(query_vector)
        
        similar_indices = indices[0][1:n_recommendations+1]
        
        return self.df.iloc[similar_indices][['Product_ID', 'Product_Type', 'Brand', 'Color', 'Style', 'Price']]

    def get_outfit_recommendation(self, product_id, n_recommendations=15):
        """Kombin önerisi"""
        try:
            idx = self.df.index[self.df['Product_ID'] == product_id].tolist()[0]
            source_product = self.df.iloc[idx]
            source_type = source_product['Product_Type']
        except IndexError:
            return pd.DataFrame()

        target_types = self.complementary_map.get(source_type, ['Jeans', 'Sneakers'])
        
        if not target_types:
            return pd.DataFrame()
            
        items_per_category = max(1, n_recommendations // len(target_types)) + 1
        
        recommended_indices = []
        
        for target_type in target_types:
            target_indices = self.df.index[self.df['Product_Type'] == target_type].tolist()
            if not target_indices: continue
                
            source_vector = self.features_matrix[idx].reshape(1, -1)
            target_vectors = self.features_matrix[target_indices]
            
            k = min(len(target_indices), items_per_category)
            
            nbrs = NearestNeighbors(n_neighbors=k, metric='cosine').fit(target_vectors)
            dist, relative_idx = nbrs.kneighbors(source_vector)
            
            for i in range(k):
                best_match_idx = target_indices[relative_idx[0][i]]
                recommended_indices.append(best_match_idx)
                
        final_indices = recommended_indices[:n_recommendations]
        
        return self.df.iloc[final_indices][['Product_ID', 'Product_Type', 'Brand', 'Color', 'Style', 'Price']]

In [11]:
# ANALİZ VE ÖNERİ FONKSİYONU
def analyze_image_and_recommend(image_path, recommender_system, strict_mode=True):
    """
    1. Görseli analiz eder (API veya Mock).
    2. Benzer ürünleri bulur (15 Adet).
    3. Kombin önerileri oluşturur (15 Adet).
    """
    
    # 1. Görsel Kontrolü
    try:
        img = Image.open(image_path)
    except FileNotFoundError:
        return f"❌ Hata: '{image_path}' bulunamadı."

    extracted_features = None
    source = "Unknown"

    # 2. API / MOCK KARARI
    if API_KEY and HAS_GENAI:
        try:
            print("🌐 Canlı Gemini Analizi Başlatılıyor...")
            model = genai.GenerativeModel('gemini-2.0-flash')
            
            prompt = """
            Analyze the clothing item. Return ONLY JSON.
            Strictly use ONLY the values from these lists provided below for categorical fields.
            If uncertain, pick the closest visual match.
            
            Allowed Values:
            - Product_Type: ['T-Shirt', 'Blouse', 'Shirt', 'Tank Top', 'Hoodie', 'Sweater', 'Cardigan', 'Jacket', 'Coat', 'Blazer', 'Jeans', 'Trousers', 'Shorts', 'Skirt', 'Dress', 'Sneakers', 'Boots', 'Heels']
            - Color: ['Black', 'White', 'Red', 'Blue', 'Green', 'Yellow', 'Purple', 'Pink', 'Grey', 'Beige', 'Navy', 'Orange', 'Brown', 'Multi']
            - Pattern: ['Solid', 'Striped', 'Polka Dot', 'Floral', 'Plaid', 'Geometric', 'Animal Print', 'Checkered']
            - Style: ['Casual', 'Formal', 'Sporty', 'Bohemian', 'Vintage', 'Chic', 'Streetwear', 'Business']
            - Season: ['Spring', 'Summer', 'Autumn', 'Winter']
            - Gender: ['Men', 'Women', 'Unisex']
            """
            response = model.generate_content([prompt, img])
            json_str = response.text.strip().replace('```json', '').replace('```', '')
            extracted_features = json.loads(json_str)
            source = "🤖 Gemini AI (Canlı)"
            
            for key, val in extracted_features.items():
                if isinstance(val, list):
                    extracted_features[key] = val[0] if val else "Unknown"
            
        except Exception as e:
            print(f"⚠️ API Hatası: {e}. Offline moda geçiliyor.")
            extracted_features = None
    else:
        print("⚠️ API Key yok. Offline Mod aktif.")

    # 3. MOCK DATA (YEDEK)
    if extracted_features is None:
        file_name = os.path.basename(image_path)
        extracted_features = MOCK_RESULTS.get(file_name, MOCK_RESULTS["default"])
        source = "💾 Mock Data (Çevrimdışı)"

    print(f"✅ Kaynak: {source}")
    detected_type = extracted_features.get('Product_Type')
    detected_color = extracted_features.get('Color')
    print(f"📋 Tespit: {detected_color} {detected_type} ({extracted_features.get('Style')})")

    # 4. ML ÖNERİ MOTORU
    try:
        # A) Girdiyi Vektöre Çevir
        input_df = pd.DataFrame([extracted_features])
        
        # Eksik kolonları doldur
        for col in recommender_system.df.columns:
            if col not in input_df.columns:
                if recommender_system.df[col].dtype == 'object':
                    mode_val = recommender_system.df[col].mode()
                    input_df[col] = mode_val[0] if not mode_val.empty else "None"
                else:
                    input_df[col] = 0
        
        input_df = input_df[recommender_system.df.columns]
        query_vector = recommender_system.preprocessor.transform(input_df)
        
        # B) Benzer Ürünleri Bul
        distances, indices = recommender_system.model.kneighbors(query_vector, n_neighbors=2000)
        all_candidates = indices[0]
        
        # Kategori Filtresi
        candidates = []
        if strict_mode:
            candidates = [i for i in all_candidates if recommender_system.df.iloc[i]['Product_Type'] == detected_type]
        else:
            candidates = all_candidates.tolist()
            
        # Eğer filtre sonucu boşsa (Örn: Model 'Pants' dedi ama bizde 'Trousers' var)
        if not candidates:
            print(f"⚠️ Uyarı: '{detected_type}' kategorisinde ürün bulunamadı. Filtre gevşetiliyor.")
            candidates = all_candidates.tolist()

        # Renk Sıralaması
        exact = [i for i in candidates if recommender_system.df.iloc[i]['Color'] == detected_color]
        others = [i for i in candidates if recommender_system.df.iloc[i]['Color'] != detected_color]
        
        final_indices = (exact + others)[:15]
        similar_products = recommender_system.df.iloc[final_indices][['Product_ID', 'Product_Type', 'Brand', 'Color', 'Style', 'Price']]
        
        # C) Kombin Önerisi Al
        if not similar_products.empty:
            best_match_id = similar_products.iloc[0]['Product_ID']
            outfit_df = recommender_system.get_outfit_recommendation(best_match_id, n_recommendations=15)
        else:
            outfit_df = pd.DataFrame()
        
        return similar_products, outfit_df

    except Exception as e:
        print(f"Algoritma Hatası: {e}")
        return None, None

In [13]:
# SİSTEMİ BAŞLAT VE TEST ET

# Özel Ağırlıklarını Belirle
my_custom_weights = {
    'Product_Type': 10.0, 
    'Gender': 8.0,         
    'Style': 6.0,          
    'Color': 5.0,          
    'Season': 3.0,         
    'Price': 1.5,          
    'Brand': 0.5           
}

# Recommender'ı bu ağırlıklarla başlat
print("\n⚙️ Sistem başlatılıyor...")
recommender = FashionRecommender(df, weights=my_custom_weights)

# Test Görseli ile Analizi Başlat
print("\n📸 Görsel analiz ediliyor...")

similar_df, outfit_df = analyze_image_and_recommend("test_image.jpg", recommender)

# Sonuçları Ekrana Bas
if similar_df is not None and not similar_df.empty:
    print(f"\n✅ BENZER ÜRÜNLER (Senin için seçtiklerimiz):")
    # Pandas ayarları: Tabloyu düzgün göstermek için
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    print(similar_df.to_string(index=False))
else:
    print("\n⚠️ Benzer ürün bulunamadı.")

if outfit_df is not None and not outfit_df.empty:
    print(f"\n✨ KOMBİN ÖNERİLERİ (Bununla harika gider):")
    print(outfit_df.to_string(index=False))
else:
    print("\n⚠️ Kombin önerisi oluşturulamadı.")


⚙️ Sistem başlatılıyor...
Veriler vektörleştiriliyor...
Özellik ağırlıkları uygulanıyor...
Model eğitildi ve hazır!

📸 Görsel analiz ediliyor...
🌐 Canlı Gemini Analizi Başlatılıyor...
✅ Kaynak: 🤖 Gemini AI (Canlı)
📋 Tespit: Blue Shirt (Business)

✅ BENZER ÜRÜNLER (Senin için seçtiklerimiz):
Product_ID Product_Type    Brand Color      Style  Price
PROD_23728        Shirt  Brand_1  Blue Minimalist 101.19
PROD_26261        Shirt Brand_16  Blue     Casual 185.49
PROD_17260        Shirt Brand_15  Blue   Bohemian 397.76
PROD_13037        Shirt Brand_34  Blue   Business 476.69
PROD_13370        Shirt Brand_13  Blue Streetwear 191.06
PROD_17424        Shirt Brand_50  Blue   Bohemian 415.17
PROD_18202        Shirt Brand_26  Blue     Gothic  15.81
PROD_20603        Shirt Brand_37  Blue     Casual 396.03
PROD_10251        Shirt  Brand_4  Blue   Bohemian  15.66
PROD_24960        Shirt  Brand_8  Blue   Business 324.82
PROD_26362        Shirt Brand_47  Blue       Chic 242.80
PROD_26484        Shirt

In [14]:
# MODEL DEĞERLENDİRME

print("\n📊 MODEL PERFORMANS TESTİ BAŞLATILIYOR...\n")

# GROUND TRUTH (Gerçek Veriler - CEVAP ANAHTARI)
ground_truth = {
    # 1. Fotoğraf: Mavi Gömlek
    "test_image.jpg": {"Color": "Blue", "Product_Type": "Shirt"}, 
    
    # 2. Fotoğraf: Siyah Pantolon (Erkek manken)
    "test_image1.jpg": {"Color": "Black", "Product_Type": "Trousers"},
    
    # 3. Fotoğraf: Pembe Mont (Kadın manken - Puffer Jacket)
    "test_image2.jpg": {"Color": "Pink", "Product_Type": "Jacket"},
}

correct_count = 0
total_count = 0
results_table = []

print(f"{'DOSYA':<15} | {'GERÇEK':<20} | {'GEMINI TAHMİNİ':<20} | {'DURUM'}")
print("-" * 75)

# TEST DÖNGÜSÜ
for image_file, true_labels in ground_truth.items():
    if not os.path.exists(image_file):
        print(f"⚠️ {image_file} bulunamadı, atlanıyor.")
        continue
        
    try:
        # Gemini'ye sor
        img = Image.open(image_file)
        model = genai.GenerativeModel('gemini-2.0-flash')
        
        prompt = """
        Analyze the clothing item. Return ONLY JSON.
        Strictly use ONLY the values from lists:
        - Product_Type: ['T-Shirt', 'Blouse', 'Shirt', 'Tank Top', 'Hoodie', 'Sweater', 'Cardigan', 'Jacket', 'Coat', 'Blazer', 'Jeans', 'Trousers', 'Shorts', 'Skirt', 'Dress', 'Sneakers', 'Boots', 'Heels']
        - Color: ['Black', 'White', 'Red', 'Blue', 'Green', 'Yellow', 'Purple', 'Pink', 'Grey', 'Beige', 'Navy', 'Orange', 'Brown', 'Multi']
        """
        response = model.generate_content([prompt, img])
        json_str = response.text.strip().replace('```json', '').replace('```', '')
        prediction = json.loads(json_str)
        
        # Listeyi stringe çevirme düzeltmesi
        for key, val in prediction.items():
            if isinstance(val, list):
                prediction[key] = val[0] if val else "Unknown"

        # KARŞILAŞTIRMA
        # Hem Renk hem Kategori doğruysa doğru say.
        is_color_correct = prediction.get('Color') == true_labels['Color']
        is_type_correct = prediction.get('Product_Type') == true_labels['Product_Type']
        
        is_perfect_match = is_color_correct and is_type_correct
        
        if is_perfect_match:
            correct_count += 1
            status = "✅ BAŞARILI"
        else:
            status = "❌ HATALI"
            
        total_count += 1
        
        true_str = f"{true_labels['Color']} {true_labels['Product_Type']}"
        pred_str = f"{prediction.get('Color')} {prediction.get('Product_Type')}"
        
        print(f"{image_file:<15} | {true_str:<20} | {pred_str:<20} | {status}")

    except Exception as e:
        print(f"{image_file} analiz edilirken hata: {e}")

# 4. SONUÇ RAPORU
if total_count > 0:
    accuracy = (correct_count / total_count) * 100
    print("-" * 75)
    print(f"\n🎯 GENEL DOĞRULUK SKORU (Accuracy): %{accuracy:.1f}")
    print(f"Toplam {total_count} görsel test edildi. {correct_count} tanesi tam isabet.")
else:
    print("\n⚠️ Test edilecek görsel bulunamadı.")


📊 MODEL PERFORMANS TESTİ BAŞLATILIYOR...

DOSYA           | GERÇEK               | GEMINI TAHMİNİ       | DURUM
---------------------------------------------------------------------------
test_image.jpg  | Blue Shirt           | Blue Shirt           | ✅ BAŞARILI
test_image1.jpg | Black Trousers       | Black Trousers       | ✅ BAŞARILI
test_image2.jpg | Pink Jacket          | Pink Jacket          | ✅ BAŞARILI
---------------------------------------------------------------------------

🎯 GENEL DOĞRULUK SKORU (Accuracy): %100.0
Toplam 3 görsel test edildi. 3 tanesi tam isabet.
